In [3]:
BIZPATH='../montreal_subset/montreal_business.json'
USERPATH='../montreal_subset/montreal_users_who_reviewed.json'
REVPATH='../montreal_subset/montreal_reviews.json'
PARTIAL_DIM_PATH = '../montreal_subset/featureset1_but_votes_Restaurants.json'

In [4]:
import json

In [5]:
bizrecords = [json.loads(line) for line in open(BIZPATH)]
userrecords = [json.loads(line) for line in open(USERPATH)]
revrecords = [json.loads(line) for line in open(REVPATH)]
pdimrecords = [json.loads(line) for line in open(PARTIAL_DIM_PATH)]

FileNotFoundError: [Errno 2] No such file or directory: '../montreal_subset/montreal_business.json'

In [ ]:
from pandas import DataFrame, Series

In [ ]:
import pandas as pd; import numpy as np

In [ ]:
bizframe = DataFrame(bizrecords)
userframe = DataFrame(userrecords)
revframe = DataFrame(revrecords)
pdimframe = pd.read_json(PARTIAL_DIM_PATH)

In [ ]:
bizframe_sub = DataFrame(bizrecords, columns=['business_id', 'name', 'categories', 'review_count', 'stars'])

In [ ]:
rev_user_frame = pd.merge(revframe, userframe, on="user_id")
rev_biz_frame = pd.merge(revframe, bizframe, on="business_id")

In [ ]:
#Months yelping
import datetime
now = datetime.datetime.now()
userframe['months_yelping'] = userframe['yelping_since'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m')).map(lambda x: (now.year - x.year) * 12 + now.month - x.month)

In [ ]:
#Adapted from: http://stackoverflow.com/questions/21160134/flatten-a-column-with-value-of-type-list-while-duplicating-the-other-columns-va
#TODO: Find more idiomatic definition for rev_category_frame, preferably by removing flattened_categories.
flattened_categories = DataFrame([[i, category]
                                     for i, categories in rev_biz_frame['categories'].apply(list).iteritems()
                                         for category in categories],
                                columns=['index', 'category'])
flattened_categories.set_index('index')
rev_category_frame = rev_biz_frame.merge(flattened_categories, left_index = True, right_index = True)

In [ ]:
#Number businesses / category
#TODO: Add name to the size column; do some sanity checking on the data.
user_biz_revved_per_category_frame = rev_category_frame.groupby(['user_id', 'business_id', 'category']).size()

In [ ]:
rev_restaurants_frame = DataFrame(rev_category_frame.loc[rev_category_frame['category'] == 'Restaurants'])
rev_restaurants_frame['funny'] = rev_restaurants_frame['votes'].map(lambda x: x['funny'])
rev_restaurants_frame['useful'] = rev_restaurants_frame['votes'].map(lambda x: x['useful'])
rev_restaurants_frame['cool'] = rev_restaurants_frame['votes'].map(lambda x: x['cool'])

In [ ]:
rev_dim_peruser_frame = rev_restaurants_frame.groupby('user_id').aggregate(sum).reset_index()[['user_id', 'funny', 'useful', 'cool']]

In [ ]:
user_dim_frame = pd.merge(userframe[['user_id', 'months_yelping']], rev_dim_peruser_frame, on='user_id')
dimframe = pd.merge(pdimframe, user_dim_frame, on='user_id')

In [ ]:
dimframe.to_json('../montreal_subset/featureset1.json')